# Toxicity metrics data generation
In this notebook I generate toxicity metrics with the Detoxify library which is used to measure toxicity of texts, in our case tweets. 
This is meant as a suplementary approach to the Perpective API since we are limited by the number of queries when using it.

Make sure to install CUDA achieves at least 5x speed up.

In [ ]:
%reload_ext autoreload
%autoreload 2

import os 
import sys
import pandas as pd
import numpy as np
import plotly 
import plotly.graph_objects as go
import time
import random

import nltk
from detoxify import Detoxify
# nltk.download('stopwords')

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../../")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"

In [ ]:
import torch
# clear memory to reduce memory errors
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

# test if cuda is available, it has to be otherwise slow asf, 33 hours for 1.2 million tweets 
device = torch.device("cuda")
cuda_present = torch.cuda.is_available()
print(f"Cuda present: {cuda_present}")

# load the model
model = Detoxify('original', device="cuda")

In [ ]:
# for single predictions
model.predict("Love on the Spectrum is the cutest show on Netflix rn ðŸ¥¹ðŸ’“")

In [ ]:
# TODO remove
"""
tweets_df = pd.read_csv("./data/twitter_1_million_tweet_dump_29_12_2022.csv")
total_len = len(tweets_df.index)
tweets_df = tweets_df[tweets_df["lang"] == "en"]
print(f"Removed {total_len - len(tweets_df.index)} tweets out of {len(tweets_df.index)}, since they were not in English")

# if we don't do it, the toxicity metrics will missmatch down the line
tweets_df = tweets_df.reset_index(drop=True)
tweets_df
""";

## Generating toxicity scores for each tweet
The code below needed 19092 seconds (5.3 hours) to run the last time, with CUDA on ~1 million tweets.

In [1]:
# TODO add lemization of the sentences to the script
import spacy
from sentence_splitter import split_text_into_sentences

# this below does stemming
nlp = spacy.load('en_core_web_sm')
sentences_lemmas = []

for sentence in sentences:
    lemmas = nlp(sentence)
    lemmas = ' '.join([x.lemma_ for x in lemmas]) 
    sentences_lemmas.append(lemmas)
    
for sent, lemma in zip(sentences, sentences_lemmas):
    print(f"\"{sent}\" turns into \"{lemma}\"")
    
# this splits sentences
sentences = split_text_into_sentences(
    text='This is a paragraph. It contains several sentences. "But why," you ask?',
    language='en'
)

for sent in sentences:
    print(sent)

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# TODO move this to .py file
def generate_toxicity_for_tweet_file(input_file, output_file_name):
    print(f"\nStarted generating toxicity metrics for:\n"
      f"-input file: '{input_file}',\n"
      f"-output file: '{output_file_name}'")
    tweets_df = pd.read_csv("./data/raw_hashtags/" + input_file)
    total_len = len(tweets_df.index)
    tweets_df = tweets_df[tweets_df["lang"] == "en"]
    print(f"Removed {total_len - len(tweets_df.index)} tweets out of {len(tweets_df.index)}, since they were not in English")

    # if we don't do it, the toxicity metrics will missmatch down the line
    tweets_df = tweets_df.reset_index(drop=True)
    print("Tweet df:")
    display(tweets_df.head(5))

    # generating toxicity scores for each tweet
    start_time = time.time() 
    csv_columns = list(tweets_df.columns) + ["toxicity", "severe_toxicity", "obscene", "threat", "insult", "identity_attack"]
    toxicity_df = pd.DataFrame(columns=csv_columns)
    # save headers to file
    toxicity_df.to_csv(output_file_name)
    content_list = tweets_df["text"].to_list()
    
    # multi step - it should work fine now! You can use it and it should be a bit faster
    step = 50
    for i in range(0, len(tweets_df.index), step):
        if i % 500 == 0 and i != 0:
            print(f"At row: {i}")
            torch.cuda.empty_cache()
            toxicity_df.to_csv(output_file_name, mode='a', header=False)
            print("Cleared GPU cache and saved to file")
            toxicity_df = pd.DataFrame(columns=csv_columns)
            
        curr_tox_dict = model.predict(content_list[i:i+step])
        curr_tweet_dict = tweets_df.iloc[i:i+step].reset_index(drop=True).to_dict(orient="list")
        merged_tweet_tox = pd.merge(pd.DataFrame(curr_tweet_dict), pd.DataFrame(curr_tox_dict), 
                                    left_index=True, right_index=True)
        toxicity_df = pd.concat([toxicity_df, merged_tweet_tox], ignore_index=True)
        
    toxicity_df.to_csv(output_file_name, mode='a', header=False)
    print(f"Execution took: {time.time() - start_time:.2f} seconds")
    print(f"Finished saving to file '{output_file_name}'\n")

In [ ]:
# hashtag_files = ["vegetarian_hashtag_6_1_2023.csv", "trump_hashtag_04_01_2023.csv", "uno_hashtag_09_01_2023.csv", 
#                 "vegan_hashtag_6_1_2023.csv", "fitness_hashtag_08_01_2023.csv", "musk_hashtag_03_01_2023.csv",
#                "netflix_hashtag_08_01_2023.csv"]
hashtag_files = ["musk_hashtag_03_01_2023.csv", "netflix_hashtag_08_01_2023.csv"]

# to not override files by mistake
hash_int = random.randrange(1000)
for file_name in hashtag_files:
    output_file = f"./data/detoxify_toxicity_added_hashtags/{file_name.replace('.csv', '')}_detoxify_toxicity_{hash_int}.csv"
    generate_toxicity_for_tweet_file(file_name, output_file)

In [ ]:
# not needed anymore since the merge is done in the loop already
"""
# load the csv with the toxicity data
tox_df = pd.read_csv(csv_file_n)
tox_df = tox_df.drop("Unnamed: 0", axis=1)
tweets_df = tweets_df.reset_index(drop=True)
display(tox_df)
display(tweets_df)

merged_f = "merged_single_pred_toxicity_7_1.csv"
# merge tweets with toxicity and save it to a file
merged_toxic_df = pd.merge(tweets_df, tox_df, left_index=True, right_index=True)
merged_toxic_df.to_csv(merged_f)
display(merged_toxic_df)

# merged_toxic_df.iloc[1136439] 
"""
# testing problems with toxicity
# pd.set_option('display.max_colwidth', None)
# display(tox_df.iloc[1136439])
# display(tweets_df.iloc[1136439])